# Fine-tunning BioGPT

Notebook này thực hiện nhiệm vụ:
- Huấn luyện mô hình ngôn ngữ BioGPT với nhiệm vụ tổng hợp ghi chú y tế trên bộ mã chẩn đoán tăng cường
- Mô hình BioGPT yêu cầu nhiều tài nguyên tính toán, tối thiểu >=40GB GPU RAM

In [ ]:
!pip install -q transformers
!pip install -q sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.7 MB/s eta 0:00:00


In [ ]:
# hf_model_path = "microsoft/biogpt"
# biogpt = "BioGPT-Large"
biogpt = "BioGPT"
hf_model_path = f"microsoft/{biogpt}"

### Load tokenized dataset

#### Code chunking from from huggingface 🤗

In [ ]:
from datasets import load_dataset

chunk_dataset = load_dataset("ChanBeDu/rare_section_length_1024_tokenized")
# chunk_dataset = load_dataset("ChanBeDu/rare_section_length_tokenized")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/446 [00:00<?, ?B/s]

data/train-00000-of-00005.parquet:   0%|          | 0.00/26.4M [00:00<?, ?B/s]

data/train-00001-of-00005.parquet:   0%|          | 0.00/26.7M [00:00<?, ?B/s]

data/train-00002-of-00005.parquet:   0%|          | 0.00/26.4M [00:00<?, ?B/s]

data/train-00003-of-00005.parquet:   0%|          | 0.00/26.5M [00:00<?, ?B/s]

data/train-00004-of-00005.parquet:   0%|          | 0.00/27.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/169215 [00:00<?, ? examples/s]

### Fine-tunning

In [ ]:
# train_dataset = tokenized_mimic["train"]
# eval_dataset = tokenized_mimic["test"]

train_dataset = chunk_dataset["train"]
# eval_dataset = chunk_dataset["validation"]

# train_dataset = lm_dataset

In [ ]:
import math

def compute_total_steps(
    total_instances: int,
    per_device_batch_size: int,
    num_devices: int = 1,
    gradient_accumulation_steps: int = 1,
    num_epochs: int = 1
) -> int:
    """
    Tính tổng số training steps cho num_epochs epoch.

    Args:
        total_instances (int): Tổng số sample trong tập train.
        per_device_batch_size (int): Batch size mỗi thiết bị (GPU).
        num_devices (int): Số lượng GPU.
        gradient_accumulation_steps (int): Số bước tích lũy gradient.
        num_epochs (int): Số epoch muốn train.

    Returns:
        int: Tổng số step cần thiết.
    """
    effective_batch_size = per_device_batch_size * num_devices * gradient_accumulation_steps
    steps_per_epoch = math.ceil(total_instances / effective_batch_size)
    total_steps = steps_per_epoch * num_epochs
    return total_steps


total_instances = len(chunk_dataset['train'])
per_device_batch_size = 40
num_devices = 1           # thường là 1 nếu dùng Colab
gradient_accumulation_steps = 1
num_epochs = 2

steps = compute_total_steps(
    total_instances,
    per_device_batch_size,
    num_devices,
    gradient_accumulation_steps,
    num_epochs
)
print(total_instances, per_device_batch_size, num_devices, gradient_accumulation_steps, num_epochs, steps)

169215 40 1 1 2 8462


In [ ]:
from transformers import TrainingArguments
# training_args = TrainingArguments(
#     output_dir="biogpt-finetuned",
#     per_device_train_batch_size=4,
#     gradient_accumulation_steps=8,  # effective batch = 32
#     num_train_epochs=3,             # 3 epochs = safe for 12hr quota
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     lr_scheduler_type="cosine",
#     warmup_steps=500,
#     eval_strategy="no",       # skip eval unless needed
#     logging_steps=50,
#     fp16=True,                      # T4 supports mixed precision
#     dataloader_num_workers=2,
#     push_to_hub=False,
#     optim="adamw_torch_fused",      # if Torch 2.0+ for speed
# )

# training_args = TrainingArguments(
#     output_dir="biogpt-a100",
#     per_device_train_batch_size=4,       # nhỏ, có thể tăng nếu GPU còn trống
#     gradient_accumulation_steps=8,       # effective batch size = 32
#     num_train_epochs=3,                  # hợp lý
#     learning_rate=3e-5,                  # hơi cao cho pretrain-style task
#     warmup_ratio=0.03,                   # chuẩn
#     weight_decay=0.01,                   # ổn
#     lr_scheduler_type="cosine",          # tốt
#     logging_steps=25,
#     save_strategy="epoch",
#     evaluation_strategy="no",            # hợp lý cho pretraining
#     bf16=True,                           # nên dùng với A100
#     gradient_checkpointing=True,         # tiết kiệm VRAM
#     report_to="none",
#     dataloader_num_workers=4,
# )

training_args = TrainingArguments(
    output_dir=f"{biogpt}-sec-synthesis-b{per_device_batch_size}",
    per_device_train_batch_size=per_device_batch_size,             # A100 đủ sức với bf16 + checkpoint
    gradient_accumulation_steps=gradient_accumulation_steps,             # effective batch size = 36
    max_steps=steps,                           # 120M tokens, ~10k–12k steps đủ cho fine-tune
    learning_rate=1e-5,                        # giảm để fine-tune ổn định hơn
    warmup_ratio=0.05,                         # tăng nhẹ cho stability
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    logging_steps=200,
    save_strategy="steps",                     # lưu mỗi N steps thay vì epoch
    save_total_limit=4,
    save_steps=1000,
    save_safetensors=False,
    # eval_strategy="steps",                     # nên có eval
    # eval_steps=10,                           # mỗi lần save thì eval luôn
    eval_strategy="no",
    bf16=True,
    gradient_checkpointing=True,
    report_to="none",
    dataloader_num_workers=4,
    push_to_hub=True
)

ValueError: Your setup doesn't support bf16/gpu.

Training from resume, push to hub on step

In [ ]:
from transformers import Trainer, AutoModelForCausalLM, AutoTokenizer
import os
from transformers import DataCollatorForLanguageModeling


output_dir = training_args.output_dir
checkpoint_dir = None

# Nếu có checkpoint trước đó thì resume
if os.path.isdir(output_dir):
    checkpoints = [os.path.join(output_dir, d) for d in os.listdir(output_dir) if d.startswith("checkpoint")]
    if checkpoints:
        checkpoint_dir = sorted(checkpoints, key=os.path.getmtime)[-1]

model_name = hf_model_path
model = AutoModelForCausalLM.from_pretrained(checkpoint_dir or model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

NameError: name 'training_args' is not defined

#### Testing dataset

In [ ]:
print(len(chunk_dataset['train']))
print(len(chunk_dataset['train'][0]['input_ids']))

# Decode to text
decoded_text = tokenizer.decode(chunk_dataset["train"][0]['input_ids'], skip_special_tokens=True)
decoded_text2 = tokenizer.decode(chunk_dataset["train"][1]['input_ids'], skip_special_tokens=True)
decoded_text3 = tokenizer.decode(chunk_dataset["train"][2]['input_ids'], skip_special_tokens=True)
decoded_text4 = tokenizer.decode(chunk_dataset["train"][3]['input_ids'], skip_special_tokens=True)
decoded_text5 = tokenizer.decode(chunk_dataset["train"][4]['input_ids'], skip_special_tokens=True)
decoded_text6 = tokenizer.decode(chunk_dataset["train"][5]['input_ids'], skip_special_tokens=True)
decoded_text7 = tokenizer.decode(chunk_dataset["train"][6]['input_ids'], skip_special_tokens=True)

print(len(chunk_dataset["train"][0]['input_ids']))
print(len(chunk_dataset["train"][1]['input_ids']))
print(len(chunk_dataset["train"][2]['input_ids']))
print(len(chunk_dataset["train"][3]['input_ids']))
print(len(chunk_dataset["train"][4]['input_ids']))
print(len(chunk_dataset["train"][5]['input_ids']))
print(decoded_text)
print(decoded_text2)
print(decoded_text3)
print(decoded_text4)
print(decoded_text5)
print(decoded_text6)
print(decoded_text7)

In [ ]:
from transformers import  AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(hf_model_path)

In [ ]:
bid = 0
print(tokenizer.decode(chunk_dataset["train"][bid]['input_ids'], skip_special_tokens=True))
tokens = tokenizer.convert_ids_to_tokens(chunk_dataset["train"][bid]['input_ids'])
print(tokens)
print(chunk_dataset["train"][bid]['input_ids'])

Diagnosis Codes: 585.6 (End stage renal disease); 038.9 (Unspecified septicemia); 427.31 (Atrial fibrillation); 078.5 (Cytomegaloviral disease); 242.90 (Thyrotoxicosis without mention of goiter or other cause, and without mention of thyrotoxic crisis or storm); 710.0 (Systemic lupus erythematosus); 453.82 (Acute venous embolism and thrombosis of deep veins of upper extremity) Procedure Codes: None Discharge Disposition: Extended Care Facility: [* * Hospital6 1643 * *]
['Diagnosis</w>', 'C', 'odes</w>', ':</w>', '58', '5.6</w>', '(</w>', 'End</w>', 'stage</w>', 'renal</w>', 'disease</w>', ')</w>', ';</w>', '0', '38.9</w>', '(</w>', 'Un', 'specified</w>', 'septicemia</w>', ')</w>', ';</w>', '42', '7.', '31</w>', '(</w>', 'Atrial</w>', 'fibrillation</w>', ')</w>', ';</w>', '0', '7', '8.5</w>', '(</w>', 'Cyt', 'omegal', 'o', 'viral</w>', 'disease</w>', ')</w>', ';</w>', '24', '2.', '90</w>', '(</w>', 'Thyro', 'toxicosis</w>', 'without</w>', 'mention</w>', 'of</w>', 'goiter</w>', 'or</w>', 

In [ ]:
bid = 1
print(tokenizer.decode(chunk_dataset["train"][bid]['input_ids'], skip_special_tokens=True))
tokens = tokenizer.convert_ids_to_tokens(chunk_dataset["train"][bid]['input_ids'])
print(tokens)
print(chunk_dataset["train"][bid]['input_ids'])

Diagnosis Codes: 585.6 (End stage renal disease); 038.9 (Unspecified septicemia); 427.31 (Atrial fibrillation); 078.5 (Cytomegaloviral disease); 242.90 (Thyrotoxicosis without mention of goiter or other cause, and without mention of thyrotoxic crisis or storm); 710.0 (Systemic lupus erythematosus); 453.82 (Acute venous embolism and thrombosis of deep veins of upper extremity) Procedure Codes: None Brief Hospital Course: 52 year old female with ESRD on HD, recent VRE bacteremia, CMV Viremia, SLE presented with fever and hypotension, developed Afib with RVR as well as labile t wave inversion, now hemodynamically stable. # EARLY SEPSIS: Patient presented with fevers, hyotension, tachycardia and a lactate of 4.8. In addition, her WBC was 4.0 but with an 8% bandemia. She has had a number of infections recently in the setting of immunosuppression. The differential was broad including line infection (new HD line placed on [* * 12-31 * *]), pneumonia (CXR without obvious infiltrate), CMV Virem

In [ ]:
bid = 2
print(tokenizer.decode(chunk_dataset["train"][bid]['input_ids'], skip_special_tokens=True))
tokens = tokenizer.convert_ids_to_tokens(chunk_dataset["train"][bid]['input_ids'])
print(tokens)
print(chunk_dataset["train"][bid]['input_ids'])

HD on [* * 1-17 * *] that then turned upright. It was not clear that the T wave inversions were rate related. Cardiology was [* * Month / Year (2) 4221 * *]. The ddx included: ischemia, Takotsubo's, or a cerebral processes, however rapid resolution of the T waves made the later two less likely. She denied chest discomfort though she occasionally had nausea. She did not have any neurological symptoms. Patient has no LVH on prior ECHOs to invoke repolarization changes. Recommend performing persantine study to r / o ischemia as an outpatient, not initiated as an inpatient given difficulty to instigate intervention in this setting with recent bacteremia and RUE thrombus. In the mean time, patient is medically managed for coronary artery disease; she is on aspirin and small dose of beta-blocker. Simvastatin was added during this admission.. # Tachycardia: In addition to atrial fibrillation which is currently controlled, she had multiple episodes of regular tachycardia. EKG revealed sinus ta

In [ ]:
bid = 3
print(tokenizer.decode(chunk_dataset["train"][bid]['input_ids'], skip_special_tokens=True))
tokens = tokenizer.convert_ids_to_tokens(chunk_dataset["train"][bid]['input_ids'])
print(tokens)

Diagnosis Codes: 585.6 (End stage renal disease); 038.9 (Unspecified septicemia); 427.31 (Atrial fibrillation); 078.5 (Cytomegaloviral disease); 242.90 (Thyrotoxicosis without mention of goiter or other cause, and without mention of thyrotoxic crisis or storm); 710.0 (Systemic lupus erythematosus); 453.82 (Acute venous embolism and thrombosis of deep veins of upper extremity) Procedure Codes: None Major Surgical or Invasive Procedure: Hemodialysis
['Diagnosis</w>', 'C', 'odes</w>', ':</w>', '58', '5.6</w>', '(</w>', 'End</w>', 'stage</w>', 'renal</w>', 'disease</w>', ')</w>', ';</w>', '0', '38.9</w>', '(</w>', 'Un', 'specified</w>', 'septicemia</w>', ')</w>', ';</w>', '42', '7.', '31</w>', '(</w>', 'Atrial</w>', 'fibrillation</w>', ')</w>', ';</w>', '0', '7', '8.5</w>', '(</w>', 'Cyt', 'omegal', 'o', 'viral</w>', 'disease</w>', ')</w>', ';</w>', '24', '2.', '90</w>', '(</w>', 'Thyro', 'toxicosis</w>', 'without</w>', 'mention</w>', 'of</w>', 'goiter</w>', 'or</w>', 'other</w>', 'cause</

#### Train

In [ ]:
from huggingface_hub import login

login(token="")  # Get this from https://huggingface.co/settings/tokens

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,    # Dataset bạn chuẩn bị
    # eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/tmp/ipython-input-2789636750.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train(resume_from_checkpoint=checkpoint_dir)

`use_cache=True` is incompatible with gradient checkpointing`. Setting `use_cache=False`...


Step,Training Loss
200,2.906100
400,1.973100
600,1.602200
800,1.452000
1000,1.380700
1200,1.322400
1400,1.299400
1600,1.271400
1800,1.228500
2000,1.210800


TrainOutput(global_step=8462, training_loss=1.2213187103884577, metrics={'train_runtime': 11193.4589, 'train_samples_per_second': 30.239, 'train_steps_per_second': 0.756, 'total_flos': 6.286003520982221e+17, 'train_loss': 1.2213187103884577, 'epoch': 2.0})

In [ ]:
import torch
import gc

def clear_gpu_memory():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    print("✅ GPU memory cleared.")
clear_gpu_memory()

In [ ]:
# import math

# eval_results = trainer.evaluate()
# print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
# from huggingface_hub import login

# login(token="")  # Get this from https://huggingface.co/settings/tokens
# trainer.push_to_hub()

##### Utils: deletet trained folder (WARNING: RUN WITH CAUTION)

In [ ]:
# import shutil
# import os

# def delete_folder_recursively(folder_path):
#     """
#     Xóa toàn bộ thư mục và các file bên trong một cách đệ quy.

#     Args:
#         folder_path (str): Đường dẫn đến thư mục cần xóa.
#     """
#     if os.path.exists(folder_path):
#         shutil.rmtree(folder_path)
#         print(f"✅ Đã xóa thư mục: {folder_path}")
#     else:
#         print(f"⚠️ Thư mục không tồn tại: {folder_path}")
